In [10]:
from skimage.filters import threshold_local
import numpy as np
import argparse
import cv2
import imutils
from matplotlib.path import Path

In [82]:
# load the image and compute the ratio of the old height
# to the new height, clone it, and resize it
image = cv2.imread(r"INSERT IMAGE PATH HERE")
image[np.where((image == [255,255,255]).all(axis = 2))] = [255,0,0]
ratio = image.shape[0] / 500.0
orig = image.copy()
image = imutils.resize(image, height = 500)
 
# convert the image to grayscale, blur it, and find edges
# in the image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(gray, 75, 200)
 
# show the original image and the edge detected image
print("STEP 1: Edge Detection")
cv2.imshow("Image", image)
cv2.imshow("Edged", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

STEP 1: Edge Detection


In [79]:
def find_center(four_points):
    x_center = 0
    y_center = 0
#     add all X points together and Y points together
    for x in four_points:
        x_center = x_center + x[0][0]
        y_center = y_center + x[0][1]
#     find average of X and Y point totals
    x_center = x_center/4.0
    y_center = y_center/4.0
    
#     create array to adhere to coordinate form
    center_point = [x_center, y_center]
    return center_point

def find_orientation(four_points):
#     the center of square/rectangle
    point_a = find_center(four_points)
#     the midpoint between point four_point[1] and four_point[2]
    point_b = [(four_points[1][0][0] + four_points[2][0][0])/2 , (four_points[1][0][1] + four_points[2][0][1])/2]
#     the point aligned with center on edge of image
    point_c = [point_a[0],666]
#     length of side connecting point_a and point_c
    ac = 666 - point_a[1]
#     length of side connecting point_a and point_b
    ab = ((point_a[0] - point_b[0])**2 + (point_a[1] - point_b[1])**2)**.5
#     length of side connecting point_b and point_c
    bc = ((point_c[0]-point_b[0])**2 + (point_c[1]-point_b[1])**2)**.5
#     calculation of orientation at point_a
    angle_orientation = np.arccos((ab**2 + ac**2 - bc**2)/(2 * ab * ac))
    return 90 - np.rad2deg(angle_orientation)

# use to find average pixel length of square side
def find_average_side_length(four_points):
    sideLengthAverage = 0
    
    sideA = ((four_points[0][0][0] - four_points[1][0][0])**2 + (four_points[0][0][1] - four_points[1][0][1])**2)**.5
    
    sideB = ((four_points[1][0][0] - four_points[2][0][0])**2 + (four_points[1][0][1] - four_points[2][0][1])**2)**.5
    
    sideC = ((four_points[2][0][0] - four_points[3][0][0])**2 + (four_points[2][0][1] - four_points[3][0][1])**2)**.5
    
    sideD = ((four_points[3][0][0] - four_points[0][0][0])**2 + (four_points[3][0][1] - four_points[0][0][1])**2)**.5
    
    sideLengthAverage = (sideA + sideB + sideC + sideD)/4
    
    return sideLengthAverage

# find center accuracy in microns
def find_center_accuracy(pixelLength, physicalLength, numPoints):
    inchesPerPixel = 0
    micronsPerPixel = 0
    
    inchesPerPixel = physicalLength / pixelLength
    
    micronsPerPixel = inchesPerPixel * 25400/(numPoints ** .5)
    
    return micronsPerPixel
    
    
# find center using many points in bounded box
def find_center_multi(image,cnts):
    dividend = 0.0
    totalx = 0.0
    totaly = 0.0
    one = 1.0
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if cv2.pointPolygonTest(cnts[0], (j,i), False) == 1:
                totalx = totalx + i
                totaly = totaly + j
                dividend = dividend + one
    center = [totaly / dividend, totalx / dividend, dividend]
    return center

In [83]:
# find the contours in the edged image, keeping only the
# largest ones, and initialize the screen contour
cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
cnts = cnts[0] if imutils.is_cv2() else cnts[1]
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
 
# loop over the contours
for c in cnts:
    # approximate the contour
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
 
    # if our approximated contour has four points, then we
    # can assume that we have found our screen
    if len(approx) == 4:
        screenCnt = approx
        break

# center of square, orientation, and more useful information
center = find_center_multi(image,cnts)
print("4 points of square", screenCnt)
print("Center using 4 corner points", find_center(screenCnt))
print("Center using points inside sqaure and number of points used", find_center_multi(image, cnts))
print("Angle of Orientation", find_orientation(screenCnt))
print("Center in microns", find_center_accuracy(find_average_side_length(screenCnt),2,center[2]))
print("STEP 2: Find contours of paper and center of paper")
print(image.size)
cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
cv2.imshow("Outline", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

('4 points of square', array([[[254, 170]],

       [[218, 180]],

       [[228, 217]],

       [[265, 206]]]))
('Center using 4 corner points', [241.25, 193.25])
('Center using points inside sqaure and number of points used', [241.07718814610612, 193.39972432804961, 1451.0])
('Angle of Orientation', 14.588918732874532)
('Center in microns', 35.11029353600984)
STEP 2: Find contours of paper and center of paper
999000
